In [6]:
#率変化(MGG(ルーレット))
import random
import math
import numpy as np
import pandas as pd
type(np.int32(np.atleast_1d(np.int32(0))))
from operator import attrgetter
wavedate=np.loadtxt('2016-09-18-16.csv', delimiter = ",", dtype = "float")#pd.read_csv("2017-01-08-14.csv", encoding="shift_jis")#
#print(wavedate.size)
#print(wavedate[:26])

def main():
    n_gene   = 52   # The number of genes.
    n_ind    = 50  #the number of individuals in a population.
    CXPB     = 0.85   # The probability of crossover.
    MUTPB    = 1.0   # The probability of individdual mutation.
    NGEN     = 40000   # The number of generation loop.
    n_parents=2     #非復元抽出の個体数
    MUTINDPB = 0.0050  # The probability of gene mutation.
    CXPB_MAX =0.900   #交叉率の最大値
    CXPB_MIN =0.200   #交叉率の最小値
    M_MAX    =10.0/52.0  #突然変異率の最大値
    M_MIN    =1.0/52.0/2  #突然変異率の最小値
    f        =15.0

    random.seed()
    # --- Step1 : 母集団作成.
    pop = create_pop(n_ind, n_gene)
    set_fitness(evalOneMax, pop)  #各個体について評価
    fit_sum = keisan(n_ind,pop)  #適応度の合計値を計算
    best_ind = min(pop, key=attrgetter("fitness"))  #最良個体の適応度

    # --- 世代分の繰り返し.
    print("Generation loop start.")
    print("Generation: 0. Best fitness: " + str(best_ind.fitness))
    for g in range(NGEN):     
        # --- Step2 : 選択.
        #elitism = []
        #elitism.append(min(pop, key=attrgetter("fitness")))
        #elite=elitism[:]
        parents,pop = randomselection(n_ind,pop,n_parents)
        fit_sum=fit_sum-parents[0].fitness-parents[1].fitness
        #set_fitness(evalOneMax, parents)
        children = parents
        #worst_ind = max(pop, key=attrgetter("fitness"))
        #C_max = worst_ind.fitness
        #offspring,fitness_mean,fit_var = roulette(pop, n_ind,C_max)
        #print("offspring1"+str(len(offspring)))
        #offspring = tournament(pop, n_ind,tournsize)
  
        # --- Step3 : 交叉.
        crossover = []
        for child1, child2 in zip(children[::2], children[1::2]):
            if random.random() < CXPB:
                child1, child2 = cxTwoPointCopy(child1, child2)
                child1.fitness = None
                child2.fitness = None
            crossover.append(child1)
            crossover.append(child2)
        #crossover.append(min(elite, key=attrgetter("fitness")))
        children = crossover[:]
        #print("offspring2"+str(len(offspring)))

        # --- Step4 : 突然変異.
        mutant = []
        for mut in (children):
            if random.random() < MUTPB:
                mut = mutFlipBit(mut, indpb=MUTINDPB)
                mut.fitness = None
            mutant.append(mut)
        #mutants=mutant[:]
        #print("mutants"+str(len(mutants)))
        #mutant.append(min(elite, key=attrgetter("fitness")))
        children = mutant[:]
        set_fitness(evalOneMax, children)
        #print("offspring3"+str(len(offspring)))
        
         #-----Step5 : 生存選択
        family = children+parents
        offspring = []
        #for i in range(int(n_parents/2)):
            #worst_ind = max(family, key=attrgetter("fitness"))
            #C_max     = worst_ind.fitness
            #offspring = roulette(family,C_max)
            #offspring.append(random.choice(family))
        #family_sorted = sorted(family, key=attrgetter("fitness"))
        #offspring = offspring+family_sorted[:int(n_parents/2)]
        
        family_sorted = sorted(family, key=attrgetter("fitness"))
        for i in range(int(n_parents/2)):
            worst_ind = max(family_sorted[int(n_parents/2):], key=attrgetter("fitness"))
            C_max     = worst_ind.fitness
            offspring = roulette(family_sorted[int(n_parents/2):],C_max)
            #offspring.append(random.choice(family))
        offspring = offspring+family_sorted[:int(n_parents/2)]

        
        # --- 次世代を現世代に更新.
        pop = pop + offspring
        fit_sum = fit_sum+offspring[0].fitness+offspring[1].fitness
        fitness_mean =fit_sum/n_gene #,fit_var
        # --- Print best fitness in the population.
        best_ind = min(pop, key=attrgetter("fitness"))
        f_max = best_ind.fitness
        CXPB,MUTINDPB = adjustrating(CXPB,MUTINDPB,CXPB_MAX,CXPB_MIN,M_MAX,M_MIN,f,f_max,fitness_mean)
        print("Generation: " + str(g+1) + ". Best fitness: " + str(best_ind.fitness)+ " mean: "+str(int(fitness_mean))+" MU: "+str(MUTINDPB))#+" CXPB: "+str(CXPB))  #+" variance: " + str(int(fit_var))
        #if best_ind.fitness <= 0.0 :
            #break
    lambda_1,theta_1,omega_1, Hv_1,lambda_2,theta_2,omega_2,Hv_2=get_parameter_from_ind(best_ind)   
    print("Generation loop ended. The best individual: ")
    print(best_ind)
    print("Hv1",Hv_1,"theta1",theta_1,"lambda1",lambda_1,"w1",omega_1)
    print("Hv2",Hv_2,"theta2",theta_2,"lambda2",lambda_2,"w2",omega_2)
   
class Individual(np.ndarray):
    """Container of a individual."""
    fitness = None
    
    def __new__(cls, a):
        return np.asarray(a).view(cls)

def create_ind(n_gene):
    """Create a individual."""
    return Individual([random.randint(0, 1) for i in range(n_gene)])

def create_pop(n_ind, n_gene):
    """Create a population."""
    pop = []
    for i in range(n_ind):
        ind = create_ind(n_gene)
        pop.append(ind)
    return pop

def set_fitness(eval_func, pop):
    """Set fitnesses of each individual in a population."""
    for i, fit in zip(range(len(pop)), map(eval_func, pop)):
        pop[i].fitness = fit
    
def evalOneMax(ind):
    """Objective function."""
    errorfunc=0.0
    lambda_1,theta_1,omega_1, Hv_1,lambda_2,theta_2,omega_2,Hv_2 = get_parameter_from_ind(ind)
   
    for j in range(36):
        theta_func=wavedate[0][j+1]
        for i in range(25):
            omega_func=wavedate[i+1][0]*2*math.pi
            HL1=Hv_1/(1.56*4*math.pi**2/omega_1**2)
            """波集中度s1の周波数依存"""
            if HL1<0.026:
                if HL1>=0.009:
                    S_max1=12.99*10**(0.4444*math.log10(HL1)*(-2.5)+0.5849*(-2.5))
                else:
                    S_max1=84
            elif HL1>=0.026:
                if HL1<=0.05:
                    S_max1=10**(-3.913*math.log10(HL1)-4.79)
                else:
                    S_max1=1.98 
    
            if omega_func<=omega_1:
                s_1=math.ceil(S_max1*omega_func**5/omega_1**5)
            else:
                s_1=math.ceil(S_max1*omega_func**(-2.5)/omega_1**(-2.5))
            
            """波集中度s2の周波数依存"""
            HL2=Hv_2/(1.56*4*math.pi**2/omega_2**2)
            if HL2<0.026:
                if HL2>=0.009:
                    S_max2=12.99*10**(0.4444*math.log10(HL2)*(-2.5)+0.5849*(-2.5))
                else:
                    S_max2=84
            elif HL2>=0.026:
                if HL2<=0.05:
                    S_max2=10**(-3.913*math.log10(HL2)-4.79)
                else:
                    S_max2=1.98      
            
            if omega_func<=omega_2:
                s_2=math.ceil(S_max2*omega_func**5/omega_2**5)
            else:
                s_2=math.ceil(S_max2*omega_func**(-2.5)/omega_2**(-2.5))

            spectrum_11=0.250*((4.0*lambda_1+1.0)/4.0*omega_1**4.0)**lambda_1
            spectrum_12=math.gamma(lambda_1)
            spectrum_16=Hv_1**2.0
            spectrum_13=omega_func**(4.0*lambda_1+1.0)
            spectrum_14=(math.gamma(s_1+1.0))**2.0
            spectrum_18=math.pi*math.gamma(2.0*s_1+1.0)
            spectrum_17=2.0**(2.0*s_1-1.0)
            spectrum_15=abs((math.cos((theta_func-theta_1)/2.0*math.pi/180.0)))**(2.0*s_1)*math.exp(-(4.0*lambda_1+1.0)/4.0*(omega_1/omega_func)**4.0)
            spectrum_1=spectrum_11/spectrum_13*spectrum_14*spectrum_15*spectrum_16/spectrum_12*spectrum_17/spectrum_18
            spectrum_21=0.250*((4.0*lambda_2+1.0)/4.0*omega_2**4.0)**lambda_2
            spectrum_22=math.gamma(lambda_2)
            spectrum_26=Hv_2**2.0
            spectrum_23=omega_func**(4.0*lambda_2+1.0)
            spectrum_24=(math.gamma(s_2+1.0))**2.0
            spectrum_28=math.pi*math.gamma(2.0*s_2+1.0)
            spectrum_27=2.0**(2.0*s_2-1.0)
            spectrum_25=abs((math.cos((theta_func-theta_2)/2.0*math.pi/180.0)))**(2.0*s_2)*math.exp(-(4.0*lambda_2+1.0)/4.0*(omega_2/omega_func)**4.0)
            spectrum_2=spectrum_21/spectrum_23*spectrum_24*spectrum_25*spectrum_26/spectrum_22*spectrum_27/spectrum_28
            if i==0:
                df=wavedate[2][0]-wavedate[1][0]
            elif i==24:
                df=wavedate[24][0]-wavedate[23][0]
            else:
                df=(wavedate[i+1][0]-wavedate[i-1][0])/2
            E=((wavedate[i+1][j+1]-2*math.pi*(spectrum_1+spectrum_2)))**2.0
            if wavedate[i+1][j+1]>2*math.pi*(spectrum_1+spectrum_2):
                errorfunc +=E#+E*wavedate[i+1][j+1]
            else:
                errorfunc +=E#+E*2*math.pi*(spectrum_1+spectrum_2)
    return errorfunc

def adjustrating(C_p,M_p,CXPB_MAX,CXPB_MIN,M_MAX,M_MIN,f,f_max,fitness_mean):
    cross_para = 1.0/f-1/abs(f_max-fitness_mean)
    if (C_p*(1+cross_para))<=CXPB_MIN:
        C_p = CXPB_MIN
    else:
        if (C_p*(1+cross_para))<=CXPB_MAX:
            C_p = C_p*(1+cross_para)
        else:
            C_p = CXPB_MAX       
                                  
    M_para = 1/abs(f_max-fitness_mean) - 1/f
    if (M_p*(1+M_para))<=M_MIN:
        M_p = M_MIN
    else:
        if (M_p*(1+M_para))<=M_MAX:
            M_p = M_p*(1+M_para)
        else:
            M_p = M_MAX
         
    return  C_p,M_p

def randomselection(n_ind,pop,n_parents):
    chosen=[]    
    for i in range(n_parents):
        idx=random.randrange(0, n_ind-i)
        chosen.append(pop[idx])
        pop.pop(idx)
    return chosen,pop
    """chosen = []   
    eve1=random.randrange(0, n_ind, 2)
    odd1=random.randrange(1, n_ind, 2)
    chosen.append(pop[eve1])
    chosen.append(pop[odd1])
    if eve1>odd1:
        pop.pop(eve1)
        pop.pop(odd1)
    else:
        pop.pop(odd1)
        pop.pop(eve1) 
    return chosen,pop"""


def tournament(pop, n_ind, tournsize):
    """#Selection function."""
    chosen = []    
    for i in range(n_ind):
        aspirants = [random.choice(pop) for j in range(tournsize)]
        chosen.append(min(aspirants, key=attrgetter("fitness")))
    return chosen

def roulette(pop,C_max):
    """Selection function."""
    chosen=[]
    fitness_sum=0.0
    for i in range(len(pop)):
        fitness_sum += C_max- pop[i].fitness
        
    sigma_fitness = 0.0
    s = random.random()*fitness_sum
    for i in range(len(pop)):
        sigma_fitness += C_max-pop[i].fitness
        if (sigma_fitness)>=s:
            chosen.append(pop[i])
            break
    #chosens=chosen[:]
    #print("chosens"+str(len(chosens)))
    return chosen

def cxTwoPointCopy(ind1, ind2):
    """Crossover function."""
    size = len(ind1)
    tmp1 = ind1.copy()
    tmp2 = ind2.copy()
    cxpoint1 = random.randint(1, size)
    cxpoint2 = random.randint(1, size-1)
    if cxpoint2 >= cxpoint1:
        cxpoint2 += 1
    else: # Swap the two cx points
        cxpoint1, cxpoint2 = cxpoint2, cxpoint1
    tmp1[cxpoint1:cxpoint2], tmp2[cxpoint1:cxpoint2] = tmp2[cxpoint1:cxpoint2].copy(), tmp1[cxpoint1:cxpoint2].copy()
    return tmp1, tmp2

def evalchild(offspring,n_ind):
    fit_mean = 0.0
    fit_var  = 0.0
    for i in range(n_ind):
        fit_mean += offspring[i].fitness/n_ind
    for k in range(n_ind):
        fit_var += (offspring[k].fitness - fit_mean)**2/n_ind
    return fit_mean, fit_var

def mutFlipBit(ind, indpb):
    """Mutation function."""
    tmp = ind.copy()
    for i in range(len(ind)):
        if random.random() < indpb:
            tmp[i] = type(ind[i])(not ind[i])
    return tmp

def keisan(n_ind,pop):
    Ex=0.0
    #Var=0.0
    for i in range(n_ind):
        Ex += pop[i].fitness
    #for i in range(n_ind):
        #Var += (pop[i].fitness-Ex)**2/n_ind
    return Ex#,Var
        
def get_parameter_from_ind2(ind):
    lambda_1 = (1.0*ind[0]+2*ind[10]+4*ind[20]+8*ind[30]+16*ind[40])*0.10+0.10
    theta_1  = (1.0*ind[1]+2*ind[11]+4*ind[21]+8*ind[31]+16*ind[41]+32*ind[50]+64*ind[56])*2.8125
    s_1      = (1.0*ind[2]+2*ind[12]+4*ind[22]+8*ind[32]+16*ind[42])+1.0
    omega_1  = (1.0*ind[3]+2*ind[13]+4*ind[23]+8*ind[33]+16*ind[43]+32*ind[51])*0.015+0.20
    Hv_1     = (1.0*ind[4]+2*ind[14]+4*ind[24]+8*ind[34]+16*ind[44]+32.0*ind[52]+64.0*ind[57]+128.0*ind[60])*0.040+1.0
    lambda_2 = (1.0*ind[5]+2*ind[15]+4*ind[25]+8*ind[35]+16*ind[45])*0.10+0.10
    theta_2  = (1.0*ind[6]+2*ind[16]+4*ind[26]+8*ind[36]+16*ind[46]+32*ind[53]+64*ind[58])*2.8125
    s_2      = (1.0*ind[7]+2*ind[17]+4*ind[27]+8*ind[37]+16*ind[47])+1.0
    omega_2  = (1.0*ind[8]+2*ind[18]+4*ind[28]+8*ind[38]+16*ind[48]+32*ind[54])*0.040+0.20
    Hv_2     = (1.0*ind[9]+2.0*ind[19]+4.0*ind[29]+8.0*ind[39]+16.0*ind[49]+32.0*ind[55]+64.0*ind[59]+128.0*ind[61])*0.040+1.0# print("Generation loop ended. The best individual: ")
   #print("Hv1",Hv_1,"s1",s_1,"theta1",theta_1,"lambda1",lambda_1,"w1",omega_1)
    #print("Hv2",Hv_2,"s2",s_2,"theta2",theta_2,"lambda2",lambda_2,"w2",omega_2)
    # print(" fitness_mean "+str(fitness_mean)+" variance: " + str(fit_var))
    return lambda_1, s_1, theta_1, omega_1, Hv_1, lambda_2, s_2, theta_2, omega_2,Hv_2

def get_parameter_from_ind(ind):
    Hv_1     =(1.0*ind[0]+2*ind[1]+4*ind[2]+8*ind[3]+16*ind[4]+32.0*ind[5]+64.0*ind[6]+128.0*ind[7])*0.020+0.2
    lambda_1 =(1.0*ind[8]+2*ind[9]+4*ind[10]+8*ind[11]+16*ind[12])*0.10+0.10
    theta_1  =(1.0*ind[13]+2*ind[14]+4*ind[15]+8*ind[16]+16*ind[17]+32*ind[18]+64*ind[19])*2.8125
    #s_1      =(1.0*ind[2]+2*ind[12]+4*ind[22]+8*ind[32]+16*ind[42])+1.0
    omega_1  =(1.0*ind[20]+2*ind[21]+4*ind[22]+8*ind[23]+16*ind[24]+32*ind[25])*0.010+0.30 
    lambda_2 =(1.0*ind[26]+2*ind[27]+4*ind[28]+8*ind[29]+16*ind[30])*0.10+0.10
    theta_2  =(1.0*ind[31]+2*ind[32]+4*ind[33]+8*ind[34]+16*ind[35]+32*ind[36]+64*ind[37])*2.8125
    #s_2      =(1.0*ind[7]+2*ind[17]+4*ind[27]+8*ind[37]+16*ind[47])+1.0
    omega_2  =(1.0*ind[38]+2*ind[39]+4*ind[40]+8*ind[41]+16*ind[42]+32*ind[43])*0.010+0.30
    Hv_2     =(1.0*ind[44]+2.0*ind[45]+4.0*ind[46]+8.0*ind[47]+16.0*ind[48]+32.0*ind[49]+64.0*ind[50]+128.0*ind[51])*0.020+0.2
    return lambda_1,theta_1,omega_1, Hv_1,lambda_2,theta_2,omega_2,Hv_2


if __name__ == "__main__":

    main()

Generation loop start.
Generation: 0. Best fitness: 1101.113333162136
Generation: 1. Best fitness: 1101.113333162136 mean: 15435 MU: 0.019230769230769232
Generation: 2. Best fitness: 1101.113333162136 mean: 15430 MU: 0.019230769230769232
Generation: 3. Best fitness: 1101.113333162136 mean: 15348 MU: 0.019230769230769232
Generation: 4. Best fitness: 1101.113333162136 mean: 15332 MU: 0.019230769230769232
Generation: 5. Best fitness: 1101.113333162136 mean: 15307 MU: 0.019230769230769232
Generation: 6. Best fitness: 1101.113333162136 mean: 15307 MU: 0.019230769230769232
Generation: 7. Best fitness: 1101.113333162136 mean: 14201 MU: 0.019230769230769232
Generation: 8. Best fitness: 1101.113333162136 mean: 14194 MU: 0.019230769230769232
Generation: 9. Best fitness: 1101.113333162136 mean: 12487 MU: 0.019230769230769232
Generation: 10. Best fitness: 1101.113333162136 mean: 12467 MU: 0.019230769230769232
Generation: 11. Best fitness: 1101.113333162136 mean: 11721 MU: 0.019230769230769232
Gene

Generation: 102. Best fitness: 771.2540412129612 mean: 1920 MU: 0.019230769230769232
Generation: 103. Best fitness: 771.2540412129612 mean: 1916 MU: 0.019230769230769232
Generation: 104. Best fitness: 771.2540412129612 mean: 1449 MU: 0.019230769230769232
Generation: 105. Best fitness: 771.2540412129612 mean: 1439 MU: 0.019230769230769232
Generation: 106. Best fitness: 771.2540412129612 mean: 1317 MU: 0.019230769230769232
Generation: 107. Best fitness: 771.2540412129612 mean: 1317 MU: 0.019230769230769232
Generation: 108. Best fitness: 771.2540412129612 mean: 1314 MU: 0.019230769230769232
Generation: 109. Best fitness: 771.2540412129612 mean: 1312 MU: 0.019230769230769232
Generation: 110. Best fitness: 771.2540412129612 mean: 1304 MU: 0.019230769230769232
Generation: 111. Best fitness: 771.2540412129612 mean: 1304 MU: 0.019230769230769232
Generation: 112. Best fitness: 771.2540412129612 mean: 1303 MU: 0.019230769230769232
Generation: 113. Best fitness: 771.2540412129612 mean: 1303 MU: 0

Generation: 206. Best fitness: 713.457407885613 mean: 868 MU: 0.019230769230769232
Generation: 207. Best fitness: 713.457407885613 mean: 857 MU: 0.019230769230769232
Generation: 208. Best fitness: 713.457407885613 mean: 856 MU: 0.019230769230769232
Generation: 209. Best fitness: 713.457407885613 mean: 856 MU: 0.019230769230769232
Generation: 210. Best fitness: 713.457407885613 mean: 857 MU: 0.019230769230769232
Generation: 211. Best fitness: 713.457407885613 mean: 863 MU: 0.019230769230769232
Generation: 212. Best fitness: 713.457407885613 mean: 856 MU: 0.019230769230769232
Generation: 213. Best fitness: 713.457407885613 mean: 855 MU: 0.019230769230769232
Generation: 214. Best fitness: 713.457407885613 mean: 855 MU: 0.019230769230769232
Generation: 215. Best fitness: 713.457407885613 mean: 855 MU: 0.019230769230769232
Generation: 216. Best fitness: 713.457407885613 mean: 856 MU: 0.019230769230769232
Generation: 217. Best fitness: 713.457407885613 mean: 856 MU: 0.019230769230769232
Gene

Generation: 314. Best fitness: 711.8248335423707 mean: 740 MU: 0.019230769230769232
Generation: 315. Best fitness: 711.8248335423707 mean: 740 MU: 0.019230769230769232
Generation: 316. Best fitness: 711.8248335423707 mean: 739 MU: 0.019230769230769232
Generation: 317. Best fitness: 711.8248335423707 mean: 739 MU: 0.019230769230769232
Generation: 318. Best fitness: 711.8248335423707 mean: 739 MU: 0.019230769230769232
Generation: 319. Best fitness: 711.8248335423707 mean: 738 MU: 0.019230769230769232
Generation: 320. Best fitness: 711.8248335423707 mean: 738 MU: 0.019230769230769232
Generation: 321. Best fitness: 711.8248335423707 mean: 738 MU: 0.019230769230769232
Generation: 322. Best fitness: 711.8248335423707 mean: 738 MU: 0.019230769230769232
Generation: 323. Best fitness: 706.9002048550334 mean: 738 MU: 0.019230769230769232
Generation: 324. Best fitness: 706.9002048550334 mean: 738 MU: 0.019230769230769232
Generation: 325. Best fitness: 706.9002048550334 mean: 738 MU: 0.01923076923

Generation: 418. Best fitness: 694.700930423555 mean: 710 MU: 0.019230769230769232
Generation: 419. Best fitness: 694.700930423555 mean: 710 MU: 0.019230769230769232
Generation: 420. Best fitness: 694.700930423555 mean: 710 MU: 0.019230769230769232
Generation: 421. Best fitness: 694.700930423555 mean: 709 MU: 0.019230769230769232
Generation: 422. Best fitness: 694.700930423555 mean: 709 MU: 0.019230769230769232
Generation: 423. Best fitness: 694.700930423555 mean: 709 MU: 0.0192405864026928
Generation: 424. Best fitness: 694.700930423555 mean: 709 MU: 0.019250408586213325
Generation: 425. Best fitness: 694.700930423555 mean: 709 MU: 0.019284581613443468
Generation: 426. Best fitness: 694.700930423555 mean: 707 MU: 0.01954270191011573
Generation: 427. Best fitness: 694.700930423555 mean: 707 MU: 0.0198287407918453
Generation: 428. Best fitness: 694.700930423555 mean: 706 MU: 0.020119951889466133
Generation: 429. Best fitness: 694.700930423555 mean: 706 MU: 0.0204154398044738
Generation:

...

Generation: 39580. Best fitness: 44.18610537127645 mean: 87 MU: 0.019230769230769232
Generation: 39581. Best fitness: 44.18610537127645 mean: 90 MU: 0.019230769230769232
Generation: 39582. Best fitness: 44.18610537127645 mean: 90 MU: 0.019230769230769232
Generation: 39583. Best fitness: 44.18610537127645 mean: 90 MU: 0.019230769230769232
Generation: 39584. Best fitness: 44.18610537127645 mean: 90 MU: 0.019230769230769232
Generation: 39585. Best fitness: 44.18610537127645 mean: 87 MU: 0.019230769230769232
Generation: 39586. Best fitness: 44.18610537127645 mean: 87 MU: 0.019230769230769232
Generation: 39587. Best fitness: 44.18610537127645 mean: 87 MU: 0.019230769230769232
Generation: 39588. Best fitness: 44.18610537127645 mean: 87 MU: 0.019230769230769232
Generation: 39589. Best fitness: 44.18610537127645 mean: 87 MU: 0.019230769230769232
Generation: 39590. Best fitness: 44.18610537127645 mean: 87 MU: 0.019230769230769232
Generation: 39591. Best fitness: 44.18610537127645 mean: 87 MU: 0

Generation: 39677. Best fitness: 44.18610537127645 mean: 47 MU: 0.024031426185204315
Generation: 39678. Best fitness: 44.18610537127645 mean: 47 MU: 0.030030491113736027
Generation: 39679. Best fitness: 44.18610537127645 mean: 45 MU: 0.04974873055324144
Generation: 39680. Best fitness: 44.18610537127645 mean: 45 MU: 0.08241410978879982
Generation: 39681. Best fitness: 44.18610537127645 mean: 45 MU: 0.13652781521754434
Generation: 39682. Best fitness: 44.18610537127645 mean: 68 MU: 0.1331401807313754
Generation: 39683. Best fitness: 44.18610537127645 mean: 68 MU: 0.1298366028705439
Generation: 39684. Best fitness: 44.18610537127645 mean: 68 MU: 0.1266149959565943
Generation: 39685. Best fitness: 44.18610537127645 mean: 68 MU: 0.12347332606254931
Generation: 39686. Best fitness: 44.18610537127645 mean: 96 MU: 0.11760083728874444
Generation: 39687. Best fitness: 44.18610537127645 mean: 98 MU: 0.11192775234339038
Generation: 39688. Best fitness: 44.18610537127645 mean: 120 MU: 0.1059256139

Generation: 39777. Best fitness: 44.18610537127645 mean: 83 MU: 0.019230769230769232
Generation: 39778. Best fitness: 44.18610537127645 mean: 101 MU: 0.019230769230769232
Generation: 39779. Best fitness: 44.18610537127645 mean: 101 MU: 0.019230769230769232
Generation: 39780. Best fitness: 44.18610537127645 mean: 101 MU: 0.019230769230769232
Generation: 39781. Best fitness: 44.18610537127645 mean: 101 MU: 0.019230769230769232
Generation: 39782. Best fitness: 44.18610537127645 mean: 101 MU: 0.019230769230769232
Generation: 39783. Best fitness: 44.18610537127645 mean: 101 MU: 0.019230769230769232
Generation: 39784. Best fitness: 44.18610537127645 mean: 101 MU: 0.019230769230769232
Generation: 39785. Best fitness: 44.18610537127645 mean: 101 MU: 0.019230769230769232
Generation: 39786. Best fitness: 44.18610537127645 mean: 101 MU: 0.019230769230769232
Generation: 39787. Best fitness: 44.18610537127645 mean: 113 MU: 0.019230769230769232
Generation: 39788. Best fitness: 44.18610537127645 mean

Generation: 39879. Best fitness: 44.18610537127645 mean: 64 MU: 0.019230769230769232
Generation: 39880. Best fitness: 44.18610537127645 mean: 64 MU: 0.019230769230769232
Generation: 39881. Best fitness: 44.18610537127645 mean: 64 MU: 0.019230769230769232
Generation: 39882. Best fitness: 44.18610537127645 mean: 64 MU: 0.019230769230769232
Generation: 39883. Best fitness: 44.18610537127645 mean: 64 MU: 0.019230769230769232
Generation: 39884. Best fitness: 44.18610537127645 mean: 64 MU: 0.019230769230769232
Generation: 39885. Best fitness: 44.18610537127645 mean: 64 MU: 0.019230769230769232
Generation: 39886. Best fitness: 44.18610537127645 mean: 64 MU: 0.019230769230769232
Generation: 39887. Best fitness: 44.18610537127645 mean: 68 MU: 0.019230769230769232
Generation: 39888. Best fitness: 44.18610537127645 mean: 68 MU: 0.019230769230769232
Generation: 39889. Best fitness: 44.18610537127645 mean: 63 MU: 0.019230769230769232
Generation: 39890. Best fitness: 44.18610537127645 mean: 60 MU: 0

Generation: 39977. Best fitness: 44.18610537127645 mean: 86 MU: 0.019230769230769232
Generation: 39978. Best fitness: 44.18610537127645 mean: 86 MU: 0.019230769230769232
Generation: 39979. Best fitness: 44.18610537127645 mean: 86 MU: 0.019230769230769232
Generation: 39980. Best fitness: 44.18610537127645 mean: 86 MU: 0.019230769230769232
Generation: 39981. Best fitness: 44.18610537127645 mean: 81 MU: 0.019230769230769232
Generation: 39982. Best fitness: 44.18610537127645 mean: 81 MU: 0.019230769230769232
Generation: 39983. Best fitness: 44.18610537127645 mean: 81 MU: 0.019230769230769232
Generation: 39984. Best fitness: 44.18610537127645 mean: 81 MU: 0.019230769230769232
Generation: 39985. Best fitness: 44.18610537127645 mean: 81 MU: 0.019230769230769232
Generation: 39986. Best fitness: 44.18610537127645 mean: 81 MU: 0.019230769230769232
Generation: 39987. Best fitness: 44.18610537127645 mean: 81 MU: 0.019230769230769232
Generation: 39988. Best fitness: 44.18610537127645 mean: 81 MU: 0